# dec-iLQR
Location for Linear-Quadratic Regulator (LQR) and iLQR for interactive trajectory planning with applications to Decentralized Control.

References:
- [1] - [Dewolf - studywolf_control](https://github.com/studywolf/control/blob/master/studywolf_control/controllers/ilqr.py)
- [2] - [Fridovich-Keil - ilqgames](https://github.com/HJReachability/ilqgames/blob/master/python/dynamical_system.py)
- [3] - [Jackson, Howell - iLQR Tutorial](http://roboticexplorationlab.org/papers/iLQR_Tutorial.pdf)
- [4] - [Jackson - AL iLQR Tutorial](https://bjack205.github.io/papers/AL_iLQR_Tutorial.pdf)
- [5] - [Sears-Collins - Linear Quadratic Regulator Python Example](https://automaticaddison.com/linear-quadratic-regulator-lqr-with-python-code-example/)

In [1]:
import sys
sys.path.append('/mnt/c/Users/ZachJW/Documents/Grad/Research/dec-ilqr/dec-ilqr/')

import numpy as np
import matplotlib.pyplot as plt

from control import iLQR, LQR
from models import DoubleInt1dDynamics, DoubleInt2dDynamics
from models import CarDynamics, UnicycleDynamics, BicycleDynamics
from dynamics import MultiDynamicalModel
from cost import ObstacleCost, ReferenceCost, CouplingCost, AgentCost
from util import Point

In [2]:
%matplotlib ipympl
np.set_printoptions(precision=3, suppress=True)

In [3]:
plt.figure()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Combining Dynamical Models

In [14]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [4]:
dt = 1.0
uni = UnicycleDynamics(dt)
car = CarDynamics(dt)
bike = BicycleDynamics(dt)
md = MultiDynamicalModel([uni, car, bike])
md.x_dims

[4, 3, 5]

In [5]:
x = np.random.randint(10, size=md.n_x)
u = np.random.randint(10, size=md.n_u)
x_next = md(x, u)
A, B = md.linearize(x, u)

In [6]:
x = np.array([0, 0, 6, 5, 
              0, 1, 5, 
            -0.5, 2, 2, 0, 4])

In [13]:
a, b = []
a.append(1)
a, b

ValueError: not enough values to unpack (expected 2, got 0)

In [7]:
pos_inds = [(0, 1), (4, 5), (7, 8)]
max_distance = 2.0
coup_cost = CouplingCost(pos_inds, max_distance)
L = coup_cost(x)
L_x, L_u, L_xx, L_uu, L_ux = coup_cost.quadraticize(x, u)

[[-2. -0.]
 [-0.  2.]]
[[0. 0.]
 [0. 0.]]
[[-0.862 -1.431]
 [-1.431  1.284]]


In [7]:
from cost import quadraticize_distance

In [29]:
pa = Point(x[pos_inds[0][0]], x[pos_inds[0][1]])
pb = Point(x[pos_inds[1][0]], x[pos_inds[1][1]])
pa, pb

((0, 0), (0, 1))

In [33]:
L_xi, L_xxi = quadraticize_distance(pa, pb, max_distance)
L_xi, L_xxi

(array([-0.,  2.]),
 array([[-2., -0.],
        [-0.,  2.]]))

### iLQR

In [4]:
dt = 0.1
N = 60
n_lqr_iter = 50
tol = 1e-3

# dynamics = DoubleInt1dDynamics(dt)
# x0 = np.array([2, 0])
# xf = np.array([0, 1])
# Q = np.diag([1, 1])
# R = np.eye(1) * 0.01

# dynamics = DoubleInt2dDynamics(dt)
# x0 = np.array([10, 10, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0])
# R = np.eye(2)

# dynamics = CarDynamics(dt)
# x0 = np.array([11, 11, np.pi/2])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 0])
# R = np.eye(2)

dynamics = UnicycleDynamics(dt)
x0 = np.array([0, 0, 0, 0])
xf = np.array([10, 10, 0, np.pi/4])
Q = np.diag([1, 1, 0, 0])
R = np.eye(2)

# dynamics = BicycleDynamics(dt)
# x0 = np.array([10, 10, np.pi/2, 0, 0])
# xf = np.array([0, 0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0, 0])
# R = np.eye(2)

Qf = np.eye(Q.shape[0]) * 1e3
reference = ReferenceCost(xf, Q, R, Qf)
obstacles = [ObstacleCost((0, 1), Point(3, 3), 2),
             ObstacleCost((0, 1), Point(9, 7), 2),
             ObstacleCost((0, 1), Point(3, 10), 3)]

REF_WEIGHT = 1.0
OBS_WEIGHT = 1e6
weights = [OBS_WEIGHT] * len(obstacles) + [REF_WEIGHT]
agent_cost = AgentCost(obstacles + [reference], weights)

In [5]:
ilqr = iLQR(dynamics, agent_cost, N=N)
X, U, Jf = ilqr.run(x0, n_lqr_iter, tol)
ilqr.plot(X, Jf, False, surface_plot=False, log_colors=True)

[run] 0/50	J: 212617
[run] 1/50	J: 66991.5	μ: 0.5	Δ: 0.5
[run] 2/50	J: 55656.4	μ: 0.125	Δ: 0.25
[run] 3/50	J: 54669.8	μ: 0.015625	Δ: 0.125
[run] 4/50	J: 54437.3	μ: 0.000976562	Δ: 0.0625
[run] 5/50	J: 54292.3	μ: 3.05176e-05	Δ: 0.03125
[run] 6/50	J: 35469	μ: 0	Δ: 0.015625
[run] 7/50	J: 14171.2	μ: 0	Δ: 0.0078125
[run] 8/50	J: 7350.45	μ: 0	Δ: 0.00390625
[run] 9/50	J: 7104.34	μ: 0	Δ: 0.00195312
[run] 10/50	J: 7023.43	μ: 0	Δ: 0.000976562
[run] 11/50	J: 3756.43	μ: 0	Δ: 0.000488281


### LQR

In [4]:
N = 60
dt = 0.1

dynamics = DoubleInt1dDynamics(dt)
x0 = np.array([10, 0])
xf = np.array([0, 0])
Q = np.diag([1, 1])
R = np.eye(1)

# dynamics = DoubleInt2dDynamics(dt)
# x0 = np.array([1, 1, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.eye(4)
# R = np.eye(2)

# dynamics = CarDynamics(dt)
# x0 = np.array([5.0, 5.0, np.pi/2])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 0])
# R = np.eye(2) * 0.01

# dynamics = UnicycleDynamics(dt)
# x0 = np.array([-1, -1, 0, np.pi/3])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0])
# R = np.eye(2)

Qf = np.eye(Q.shape[0]) * 1e3
quad_cost = ReferenceCost(xf, Q, R, Qf)
lqr = LQR(dynamics, quad_cost, N=N)
X, U, J = lqr.run(x0)
lqr.plot(X, J)

AttributeError: 'AgentCost' object has no attribute 'Q'